In [9]:
from sklearn import svm, datasets
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
iris = datasets.load_iris()

In [17]:
X = iris.data
print(X.shape)

# Add noisy features
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]
print(X.shape)

(150, 4)
(150, 804)


In [13]:
y = iris.target
y.shape

(150,)

In [23]:
y = label_binarize(y, classes=[0, 1, 2])
y.shape

(150, 3)

In [28]:
y[:, 2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [29]:
from sklearn.metrics import precision_recall_curve

In [35]:
pre, rec, ths = precision_recall_curve(
    y[:, 0],
    random_state.randn(y.shape[0])
)

In [37]:
ths.shape

(138,)

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer

In [61]:
mlb = MultiLabelBinarizer(classes=[1, 2, 3])
mlb.fit_transform([(1,), (1, 2), (1,), (2,), (1, 2)])

array([[1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 1, 0]])

In [60]:
mlb.classes_

array([1, 2])

In [64]:
precision_score(
    [[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]],
    [[1, 0, 0], [1, 1, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]],
    average='micro'
) # OK

0.2857142857142857

In [30]:
precision_recall_fscore_support(
    [[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]],
    [[1, 0, 0], [1, 1, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]],
    average='micro'
)

(0.2857142857142857, 0.4, 0.3333333333333333, None)

In [67]:
gt = np.array([[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]])
ps = np.array([[0.5, 0, 0], [0.7, 0.2, 0.1], [0.1, 0.8, 0.2], [0.78, 0, 0], [0.1, 0.1, 0.6]])

precision_recall_curve(
    gt[:, 0],
    ps[:, 0],
) # OK

(array([1., 1., 1., 1.]),
 array([1.        , 0.66666667, 0.33333333, 0.        ]),
 array([0.5 , 0.7 , 0.78]))

In [69]:
gt.ravel()

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1])

In [70]:
precision_recall_curve(
    gt.ravel(),
    ps.ravel()
) # OK

(array([1., 1., 1., 1., 1., 1.]),
 array([1. , 0.8, 0.6, 0.4, 0.2, 0. ]),
 array([0.5 , 0.6 , 0.7 , 0.78, 0.8 ]))

In [10]:
# Goal
#   - 10 thresholdovu evaluaciu
#       --> podla thresholdu urobit binarizaciu --> vypocitat 'micro' Precision/Recall/F1-Score
#       --> reportnut tieto hodnoty do tensorboardu --> dostanem 10 threshold grafov
#   - vypocet accuracy pre Classification rovnakym sposobom
#   - vypocitat AP pre detection a dat do grafu
#   - precision-recall-curve ktory ulozim ako PNG

#   - kumulativna podla thresholdu ??
#       - viem to vybrat z dat pre Precision/Recall/F1-Score z thresholdov

In [75]:
# 10 Threshold evaluaction
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

In [87]:
y_cont = []
x_cont = []
for _, values in pred.items():
    y_cont += values[GROUND_TRUTH]
    x_cont += [t[0] for t in values[PREDICTION]]
    
y_cont = np.array(y_cont)
x_cont = np.array(x_cont)

In [89]:
x_cont.shape

(272166, 43)

In [90]:
# ground-truth binarization
y_bin = label_binarize(y_cont, classes=list(range(0, 43)))

In [96]:
# calculate Precision, Recall, F1 Score
for t in np.arange(0.1, 1.0, 0.1):
    x_bin = np.where(x_cont > t, 1, 0)
    res = precision_recall_fscore_support(
        y_bin,
        x_bin,
        average='micro',
        zero_division=0
    )
    res = [round(v, 3) for v in res if v is not None]
    print(f"{round(t, 1)}, Precision: {res[0]}, Recall: {res[1]}, F1-Score: {res[2]}")

0.1, Precision: 0.318, Recall: 0.589, F1-Score: 0.413
0.2, Precision: 0.432, Recall: 0.489, F1-Score: 0.459
0.3, Precision: 0.515, Recall: 0.421, F1-Score: 0.463
0.4, Precision: 0.591, Recall: 0.367, F1-Score: 0.453
0.5, Precision: 0.663, Recall: 0.319, F1-Score: 0.431
0.6, Precision: 0.728, Recall: 0.274, F1-Score: 0.398
0.7, Precision: 0.79, Recall: 0.228, F1-Score: 0.354
0.8, Precision: 0.855, Recall: 0.166, F1-Score: 0.278
0.9, Precision: 0.942, Recall: 0.078, F1-Score: 0.144


In [93]:
ap_score = average_precision_score(
    y_bin,
    x_cont,
    average='macro'
)
print(f'\nAP: {ap_score}')


AP: 0.49069291293506195


In [1]:
####### Calculate Accuracy

In [26]:
### LOAD PREDICTIONS
import os
import pickle
import numpy as np

In [12]:
GROUND_TRUTH = "ground-truth"
PREDICTION = "prediction"
SEQ_LENGTH = "seq-length"


def default_value():
    return {
        GROUND_TRUTH: [],
        PREDICTION: [],
        SEQ_LENGTH: 0
    }

#pred = pickle.load(open('../data/predictions_2021-05-16T12:22:03.722505.p', 'rb'))

In [13]:
folder = '../output_models/cross_subject-batch_1_lr_0001_rs_overlap_2021_03_04-14_47_39/'

file_predictions = list(filter(lambda x: x.endswith('.p'), os.listdir(folder)))
file_predictions

['predictions_30_2021_05_17-19_45_15.p',
 'predictions_100_2021_05_17-20_33_50.p',
 'predictions_80_2021_05_17-20_20_06.p',
 'predictions_60_2021_05_17-20_06_05.p',
 'predictions_40_2021_05_17-19_52_10.p',
 'predictions_70_2021_05_17-20_13_03.p',
 'predictions_50_2021_05_17-19_59_03.p',
 'predictions_10_2021_05_17-19_31_14.p',
 'predictions_90_2021_05_17-20_26_58.p',
 'predictions_20_2021_05_17-19_38_10.p']

In [14]:
pfile = file_predictions[0]

data = pickle.load(open(os.path.join(folder, pfile), 'rb'))

In [15]:
type(data)

collections.defaultdict

In [16]:
data.keys()

dict_keys(['0291-L', '0291-M', '0291-R', '0292-L', '0292-M', '0292-R', '0293-L', '0293-M', '0293-R', '0294-L', '0294-M', '0294-R', '0295-L', '0295-M', '0295-R', '0296-L', '0296-M', '0296-R', '0297-L', '0297-M', '0297-R', '0298-L', '0298-M', '0298-R', '0301-L', '0301-M', '0301-R', '0302-L', '0302-M', '0302-R', '0303-L', '0303-M', '0303-R', '0304-L', '0304-M', '0304-R', '0305-L', '0305-M', '0305-R', '0306-L', '0306-M', '0306-R', '0307-L', '0307-M', '0307-R', '0308-L', '0308-M', '0308-R', '0311-L', '0311-M', '0311-R', '0312-L', '0312-M', '0312-R', '0313-L', '0313-M', '0313-R', '0314-L', '0314-M', '0314-R', '0315-L', '0315-M', '0315-R', '0316-L', '0316-M', '0316-R', '0317-L', '0317-M', '0317-R', '0318-L', '0318-M', '0318-R', '0321-L', '0321-M', '0321-R', '0322-L', '0322-M', '0322-R', '0323-L', '0323-M', '0323-R', '0324-L', '0324-M', '0324-R', '0325-L', '0325-M', '0325-R', '0326-L', '0326-M', '0326-R', '0327-L', '0327-M', '0327-R', '0328-L', '0328-M', '0328-R', '0331-L', '0331-M', '0331-R',

In [24]:
data['0291-L'].keys()

dict_keys(['ground-truth', 'prediction', 'seq-length'])

In [30]:
np.argmax(data['0291-L']['prediction'][0])

38

In [31]:
data['0291-L']['prediction'][0]

array([[2.8459190e-03, 1.1553781e-02, 7.4541368e-02, 8.0316830e-03,
        2.6821257e-03, 1.3154967e-03, 1.4384463e-04, 2.4321429e-02,
        3.9683646e-03, 7.6341722e-03, 1.6967366e-03, 1.9604236e-02,
        1.0788281e-03, 1.4819030e-03, 2.6515848e-03, 4.7615566e-03,
        7.8955851e-03, 1.9214547e-03, 2.8833842e-02, 3.6492009e-02,
        6.8346667e-03, 5.6670909e-04, 1.6281529e-03, 3.0740950e-02,
        1.4514944e-03, 1.2861119e-03, 5.3657662e-02, 6.0573067e-03,
        2.6825426e-02, 1.8227920e-02, 5.2449438e-03, 2.5670584e-03,
        3.4110632e-02, 6.2723183e-03, 7.5275227e-03, 2.2157770e-02,
        4.2854827e-03, 9.1887795e-04, 1.7055763e-01, 9.9727316e-03,
        2.4460428e-03, 8.7177136e-04, 1.9786591e-03]], dtype=float32)

In [33]:
# concatenate all predictions from all sequences
gt = []
pr = []
for _, v in data.items():
    gt += v['ground-truth']
    pr += [np.argmax(p) for p in v['prediction']]
    
print(f'L-GT: {len(gt)}')
print(f'L-PR: {len(pr)}')

L-GT: 272166
L-PR: 272166


In [35]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(gt, pr)

0.25608268483205104